In [1]:
import pandas as pd
import regex as re

In [2]:
file_path = "episode_level_measures_parts_with_viewership_new_measures.csv"
# Read the CSV file
nlpdata = pd.read_csv(file_path, sep=',', on_bad_lines='skip', index_col=False, dtype='unicode')

# Display the first few rows of the dataframe to verify the content
nlpdata.head()

,Show,Season,Episode,File Name,sd_div_mean_1,sd_div_mean_2,sd_div_mean_3,sd_sum_1,sd_sum_2,sd_sum_3,...,motion_1,motion_2,motion_3,space_1,space_2,space_3,time_1,time_2,time_3,Viewership (millions)
0,2 Broke Girls,1,01,2 Broke Girls - S01E01 - 2.Broke.Girls.Season....,5658.359420163232,9159.676336,4626.990287281845,18.66514228373773,20.545672701464397,21.69718205504922,...,1.7342857142857144,2.4214285714285713,2.0725,5.192857142857143,7.394285714285714,5.41375,4.322857142857143,3.005714285714286,2.5725,19.37
1,2 Broke Girls,1,02,2 Broke Girls - S01E02 - 2 Broke Girls S01E02 ...,6280.502095337137,3451.804293115786,7127.226693789984,19.733889186635707,20.345822809580056,17.44018977836537,...,1.4857142857142858,2.987142857142857,2.2185714285714284,6.852857142857142,7.992857142857143,6.631428571428572,4.492857142857143,4.335714285714285,3.932857142857143,11.75
2,2 Broke Girls,1,03,2 Broke Girls - S01E03 - 2 Broke Girls S01E03 ...,3931.6040952853023,3977.461758366915,5506.511934027525,21.55941993523026,20.759948099987184,19.837830499174945,...,1.741428571428571,2.3614285714285717,5.745,6.055714285714286,6.632857142857143,4.38625,4.8100000000000005,2.487142857142857,3.27375,11.42
3,2 Broke Girls,1,04,2 Broke Girls - S01E04 - 2 Broke Girls S01E04 ...,5899.921656,7097.733100232231,3386.8579172793297,21.26736255478216,20.24850059,19.43430838831536,...,2.347142857142857,3.0485714285714285,2.847142857142857,7.688571428571429,6.484285714285714,6.192857142857142,4.957142857142857,3.7471428571428578,4.097142857142857,10.71
4,2 Broke Girls,1,06,2 Broke Girls - S01E06 - 2 Broke Girls S01E06 ...,9570.497331194108,5883.807212973031,6103.310464,20.229745482690152,21.176402608924068,21.713179209464133,...,1.4257142857142855,1.9,2.015,6.475714285714285,4.26,5.53875,4.502857142857143,4.93,3.62375,11.19


In [3]:
nlpdata.dtypes

Show                     object
Season                   object
Episode                  object
File Name                object
sd_div_mean_1            object
                          ...  
space_3                  object
time_1                   object
time_2                   object
time_3                   object
Viewership (millions)    object
Length: 134, dtype: object

In [4]:
# How many TV shows do we have
len(nlpdata.Show.unique())

235

In [5]:
nlpdata.loc[:,'sd_div_mean_1':'time_3'].shape

(26088, 129)

For each show, each episode we have sentiment analysis metrics + viewership (millions) of the debut release.
Each measure appears 3 times corresponding to different acts within an episode. We have 129 measures from the sentiment analysis output / 3 acts correspond to **43 emotions per act**.

***NLP Dictionary***:
- Sd_div_mean: Aims to assess how quickly a consumer becomes immersed in a story—both mentally and emotionally. Become absorbed in the narrative. By dividing the standard deviation of overall emotion scores by the mean, this measurement captures the shifts, variability, and intensity of emotions throughout the experience.
- Sd_sum: Sum of the standard deviation, amount of emotion variance for each act.
- Sd_scaled: Standard deviation scaled across all emotions for ease of comparison.
- Anger
- Surprise
- Disgust
- Sadness
- Neutral
- Fear
- Joy
- Positive
- Negative
- Engaged: High ssychological involvement or emotional investment, including greater use of personal pronouns, more emotional words, and greater cognitive processing contained in the act.
- Not Engaged: Low psychological involvement or emotional investment, including lesser use of personal pronouns, less emotional words, and less cognitive processing contained in the act.
- WC: Total word count contained in the act.
- Analytic: Analytical, formal, or logical discussion contained in the act.
- Clout: Social status, confidence, or leadership discussion contained in the act.
- Authenticity: Honest, non-filtered, non-regulated discussion contained in the act.
- Tone: The higher the tone, the more positive the tone in the act (below 50 is considered negative).
- WPS: Average words per sentence.
- Six letter: Percentage of words longer than six letters.
- Dic: Percentage of words that were captured as dictionary words.
- Cogprocess: An aggregate measurement that looks at the amount of words that reflect active information processing and mental activities, including causation contained in the act.
    - Insight: Considers realizations.
    - Cause: Examines causation between two elements.
    - Discrep: Considers what should, could, or would have happened, but never did, (exploring counterfactuals).
    - Tentative: Looks at whether something could or could not happen (e.g. maybe, perhaps).
    - Certain: Looks at absolute language (e.g. always, never).
    - Differ (Is this differentiate?): Considers differentiation between two elements such as (hasn’t , but, else).
- Perceptual: An aggregate measurement of terms that describe perception, such as look, heard, and feeling.
    - See: Amount of text around viewing or seeing.
    - Hear: Amount of text around hearing or listening.
    - Feel: Looks at references to touch or feeling.
- Drives: An aggregate measurement that looks at different motivations.
    - Affiliation: Looks at relations and affiliations such as ally, friend, or being social.
    - Achieve: Considers the ability to win, earn success, and be better.
    - Power: Examines power dynamics and structures including superiority and bullying.
    - Reward: Examines the types of rewards that are discussed including receiving something, prizes, and benefits.
    - Risk: Examines the different types of dangers and doubts.
- Relativity: Aggregate measure extends toward spatial relationships such as area, bend, and exit.
    - Motion: Examines the ability to move, including arrive, car, and go.
    - Space: Examines directions in space, including down, and in.
    - Time: Examines time durations, including end, until and season.


# Preprocessing

In [6]:
nlpdata.head()

,Show,Season,Episode,File Name,sd_div_mean_1,sd_div_mean_2,sd_div_mean_3,sd_sum_1,sd_sum_2,sd_sum_3,...,motion_1,motion_2,motion_3,space_1,space_2,space_3,time_1,time_2,time_3,Viewership (millions)
0,2 Broke Girls,1,01,2 Broke Girls - S01E01 - 2.Broke.Girls.Season....,5658.359420163232,9159.676336,4626.990287281845,18.66514228373773,20.545672701464397,21.69718205504922,...,1.7342857142857144,2.4214285714285713,2.0725,5.192857142857143,7.394285714285714,5.41375,4.322857142857143,3.005714285714286,2.5725,19.37
1,2 Broke Girls,1,02,2 Broke Girls - S01E02 - 2 Broke Girls S01E02 ...,6280.502095337137,3451.804293115786,7127.226693789984,19.733889186635707,20.345822809580056,17.44018977836537,...,1.4857142857142858,2.987142857142857,2.2185714285714284,6.852857142857142,7.992857142857143,6.631428571428572,4.492857142857143,4.335714285714285,3.932857142857143,11.75
2,2 Broke Girls,1,03,2 Broke Girls - S01E03 - 2 Broke Girls S01E03 ...,3931.6040952853023,3977.461758366915,5506.511934027525,21.55941993523026,20.759948099987184,19.837830499174945,...,1.741428571428571,2.3614285714285717,5.745,6.055714285714286,6.632857142857143,4.38625,4.8100000000000005,2.487142857142857,3.27375,11.42
3,2 Broke Girls,1,04,2 Broke Girls - S01E04 - 2 Broke Girls S01E04 ...,5899.921656,7097.733100232231,3386.8579172793297,21.26736255478216,20.24850059,19.43430838831536,...,2.347142857142857,3.0485714285714285,2.847142857142857,7.688571428571429,6.484285714285714,6.192857142857142,4.957142857142857,3.7471428571428578,4.097142857142857,10.71
4,2 Broke Girls,1,06,2 Broke Girls - S01E06 - 2 Broke Girls S01E06 ...,9570.497331194108,5883.807212973031,6103.310464,20.229745482690152,21.176402608924068,21.713179209464133,...,1.4257142857142855,1.9,2.015,6.475714285714285,4.26,5.53875,4.502857142857143,4.93,3.62375,11.19


In [8]:
nlpdata.dtypes

Show                     object
Season                   object
Episode                  object
File Name                object
sd_div_mean_1            object
                          ...  
space_3                  object
time_1                   object
time_2                   object
time_3                   object
Viewership (millions)    object
Length: 134, dtype: object

In [9]:
# Convert Season column to integer and Show column to string
nlpdata['Season'] = nlpdata['Season'].astype(int)
nlpdata['Show'] = nlpdata['Show'].astype(str)

# Identify columns to convert to float (excluding 'Show', 'Season', and 'Episode')
columns_to_convert = nlpdata.columns.difference(['Show', 'Season', 'Episode'])
nlpdata[columns_to_convert] = nlpdata[columns_to_convert].apply(pd.to_numeric, errors='coerce')

# Drop unnecessary columns
nlpdata.drop(columns = ['File Name'], inplace=True)

# Clean 'Episode' column
nlpdata['Episode'] = (
    nlpdata['Episode']
    .str.strip()  # Remove leading/trailing whitespace
    .str.split('-')  # Take the first part before '-'
    .str[0]
    .str.extract(r'(\d+)')[0]  # Keep only digits
    .astype(int)
)

nlpdata.head()

,Show,Season,Episode,sd_div_mean_1,sd_div_mean_2,sd_div_mean_3,sd_sum_1,sd_sum_2,sd_sum_3,sd_scaled_1,...,motion_1,motion_2,motion_3,space_1,space_2,space_3,time_1,time_2,time_3,Viewership (millions)
0,2 Broke Girls,1,1,5658.359420,9159.676336,4626.990287,18.665142,20.545673,21.697182,598.633647,...,1.734286,2.421429,2.072500,5.192857,7.394286,5.413750,4.322857,3.005714,2.572500,19.37
1,2 Broke Girls,1,2,6280.502095,3451.804293,7127.226694,19.733889,20.345823,17.440190,629.842904,...,1.485714,2.987143,2.218571,6.852857,7.992857,6.631429,4.492857,4.335714,3.932857,11.75
2,2 Broke Girls,1,3,3931.604095,3977.461758,5506.511934,21.559420,20.759948,19.837830,694.243568,...,1.741429,2.361429,5.745000,6.055714,6.632857,4.386250,4.810000,2.487143,3.273750,11.42
3,2 Broke Girls,1,4,5899.921656,7097.733100,3386.857917,21.267363,20.248501,19.434308,684.387650,...,2.347143,3.048571,2.847143,7.688571,6.484286,6.192857,4.957143,3.747143,4.097143,10.71
4,2 Broke Girls,1,6,9570.497331,5883.807213,6103.310464,20.229745,21.176403,21.713179,648.841555,...,1.425714,1.900000,2.015000,6.475714,4.260000,5.538750,4.502857,4.930000,3.623750,11.19


In [10]:
# Show data
imdb_file_path = "show_info_pivoted_with_imdb (1).csv"
# Read the CSV file
imdb = pd.read_csv(imdb_file_path, sep=',', on_bad_lines='skip', index_col=False, dtype='unicode')

#View data2
imdb.head()

,Show,Executive producers,Genre,Network,No.of episodes,No.of seasons,Starring,Year
0,2 Broke Girls,Michael Patrick King | Whitney Cummings | Mich...,Sitcom,CBS,138,6,Kat Dennings | Beth Behrs | Garrett Morris | J...,2011
1,24,Joel Surnow | Robert Cochran | Brian Grazer | ...,Serial drama | Crime thriller | Espionage | Ac...,Fox,192 + 24: Redemption + 12 (24: Live Another Day),9,Kiefer Sutherland | (and | others | ),2001
2,30 Rock,Lorne Michaels | Tina Fey | Marci Klein | Davi...,Sitcom | Satire | Farce,NBC,139,7,Tina Fey | Tracy Morgan | Jane Krakowski | Jac...,2006
3,3rd Rock from the Sun,John Lithgow | Bonnie Turner | Terry Turner | ...,Sitcom | Science fiction,NBC,139,6,John Lithgow | Kristen Johnston | French Stewa...,1996
4,9/1/2001,Ryan Murphy | Brad Falchuk | Tim Minear | Juan...,Procedural drama,Fox | ABC,96,6,Angela Bassett | Peter Krause | Oliver Stark |...,2018


In [11]:
print(imdb.shape)
imdb.dtypes

(304, 8)


Show                   object
Executive producers    object
Genre                  object
Network                object
No.of episodes         object
No.of seasons          object
Starring               object
Year                   object
dtype: object

In [12]:
# Filter rows where 'Episode' contains non-numeric characters
invalid_episodes = imdb[imdb['No.of episodes'].apply(lambda x: not str(x).isdigit())]
invalid_episodes[['Show', 'No.of episodes']]

,Show,No.of episodes
1,24,192 + 24: Redemption + 12 (24: Live Another Day)
35,Brooklyn Bridge,35 (includes 2 two-part episodes)
43,Caroline in the City,97 (1 unaired)
45,Cheers,275 (including three double-length episodes an...
60,Cursed,17 (2 unaired)
72,Diagnosis: Murder,178 + 5 TV movies + Pilot
92,Family Ties,176 + one film
95,Fired Up,28 (5 unaired)
111,Grand,26 (1 unaired)
119,Hearts Afire,54 (1 unaired)


In [13]:
# Convert relevant columns to appropriate types
imdb['Year'] = imdb['Year'].astype(int)

# Define the function to truncate after the first number
def truncate_after_first_number(text):
    match = re.search(r'\d+', str(text))  # Search for the first occurrence of one or more digits
    return match.group() if match else "0"  # Return the matched number as a string or '0' if no number is found

# Step 1: Handle non-numeric entries in 'No.of seasons'
imdb['No.of seasons'] = imdb['No.of seasons'].apply(truncate_after_first_number).astype(int)
# Step 2: Handle non-numeric entries in 'No.of episodes' and convert to integer
imdb['No.of episodes'] = imdb['No.of episodes'].apply(truncate_after_first_number).astype(int)

imdb.head()

,Show,Executive producers,Genre,Network,No.of episodes,No.of seasons,Starring,Year
0,2 Broke Girls,Michael Patrick King | Whitney Cummings | Mich...,Sitcom,CBS,138,6,Kat Dennings | Beth Behrs | Garrett Morris | J...,2011
1,24,Joel Surnow | Robert Cochran | Brian Grazer | ...,Serial drama | Crime thriller | Espionage | Ac...,Fox,192,9,Kiefer Sutherland | (and | others | ),2001
2,30 Rock,Lorne Michaels | Tina Fey | Marci Klein | Davi...,Sitcom | Satire | Farce,NBC,139,7,Tina Fey | Tracy Morgan | Jane Krakowski | Jac...,2006
3,3rd Rock from the Sun,John Lithgow | Bonnie Turner | Terry Turner | ...,Sitcom | Science fiction,NBC,139,6,John Lithgow | Kristen Johnston | French Stewa...,1996
4,9/1/2001,Ryan Murphy | Brad Falchuk | Tim Minear | Juan...,Procedural drama,Fox | ABC,96,6,Angela Bassett | Peter Krause | Oliver Stark |...,2018


In [17]:
# Check again for invalid episodes
invalid_episodes = imdb[imdb['No.of episodes'].apply(lambda x: not str(x).isdigit())]
invalid_episodes[['Show', 'No.of episodes']]

,Show,No.of episodes


In [15]:
#Drop irrelevant columns
imdb.drop(columns = ["Executive producers", "Starring"], errors='ignore', axis=1, inplace=True)

# Extract the first genre before '|'
imdb['Genre'] = imdb['Genre'].str.split('|').str[0]
# Extract the first network before '|'
imdb['Network'] = imdb['Network'].str.split('|').str[0]

In [16]:
imdb.head()

,Show,Genre,Network,No.of episodes,No.of seasons,Year
0,2 Broke Girls,Sitcom,CBS,138,6,2011
1,24,Serial drama,Fox,192,9,2001
2,30 Rock,Sitcom,NBC,139,7,2006
3,3rd Rock from the Sun,Sitcom,NBC,139,6,1996
4,9/1/2001,Procedural drama,Fox,96,6,2018


In [18]:
# One-hot encoding of Network column
imdb['Network'] = imdb['Network'].str.strip()  # Clean the 'Network' column
imdb_encoded = pd.concat([imdb.drop('Network', axis=1), 
    pd.get_dummies(imdb['Network'], prefix='Network')], axis=1)

In [19]:
len(imdb.Show.unique())

304

In [20]:
imdb_encoded.head()

,Show,Genre,No.of episodes,No.of seasons,Year,Network_ABC,Network_AMC,Network_Adult Swim,Network_BBC Two,Network_CBC Television,...,Network_HBO,Network_HBO (US),Network_IFC,Network_NBC,Network_Paramount Network,Network_Showtime,Network_Starz,Network_The CW,Network_The WB,Network_USA Network
0,2 Broke Girls,Sitcom,138,6,2011,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,24,Serial drama,192,9,2001,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,30 Rock,Sitcom,139,7,2006,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,False
3,3rd Rock from the Sun,Sitcom,139,6,1996,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,False
4,9/1/2001,Procedural drama,96,6,2018,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [21]:
# Merge data and imdb_enconded dataframes on 'Show'
nlpdata_imdb = pd.merge(nlpdata, imdb_encoded, on='Show', how='left')
nlpdata_imdb.head()

,Show,Season,Episode,sd_div_mean_1,sd_div_mean_2,sd_div_mean_3,sd_sum_1,sd_sum_2,sd_sum_3,sd_scaled_1,...,Network_HBO,Network_HBO (US),Network_IFC,Network_NBC,Network_Paramount Network,Network_Showtime,Network_Starz,Network_The CW,Network_The WB,Network_USA Network
0,2 Broke Girls,1,1,5658.359420,9159.676336,4626.990287,18.665142,20.545673,21.697182,598.633647,...,False,False,False,False,False,False,False,False,False,False
1,2 Broke Girls,1,2,6280.502095,3451.804293,7127.226694,19.733889,20.345823,17.440190,629.842904,...,False,False,False,False,False,False,False,False,False,False
2,2 Broke Girls,1,3,3931.604095,3977.461758,5506.511934,21.559420,20.759948,19.837830,694.243568,...,False,False,False,False,False,False,False,False,False,False
3,2 Broke Girls,1,4,5899.921656,7097.733100,3386.857917,21.267363,20.248501,19.434308,684.387650,...,False,False,False,False,False,False,False,False,False,False
4,2 Broke Girls,1,6,9570.497331,5883.807213,6103.310464,20.229745,21.176403,21.713179,648.841555,...,False,False,False,False,False,False,False,False,False,False


In [36]:
len(nlpdata_imdb.Show.unique())

235

In [176]:
# Display a statistical summary of the 'Viewership (millions)' column
nlpdata_imdb['Viewership (millions)'].describe().T

count    25861.000000
mean        10.667185
std          7.817323
min          0.049000
25%          4.830000
50%          9.080000
75%         14.900000
max         58.530000
Name: Viewership (millions), dtype: float64

In [177]:
#Based on this, it makes sense to drop rows that have missing values
nlpdata_imdb.dropna(inplace=True)

In [41]:
file_path3 = "Air_Date1.csv"
airdate_data = pd.read_csv(file_path3, sep=',', index_col=False, dtype='unicode')

# Splitting Episode ranges into separate rows e.g. episode 21-22 into 2 separate rows
airdate_data_split = pd.concat([pd.DataFrame({
    'Show': row['Show'],
    'Season': row['Season'],
    'Episode': episode,
    'Air Date': row['Air Date']
}, index=[0]) for _, row in airdate_data.iterrows() for episode in row['Episode'].split('-') if episode.isdigit()]).reset_index(drop=True)

# Convert 'Season' and 'Episode' columns to integers
airdate_data_split[['Season','Episode']] = airdate_data_split[['Season','Episode']].astype(int)

airdate_data_split.head()

,Show,Season,Episode,Air Date
0,2 Broke Girls,1,1,19-Sep-11
1,2 Broke Girls,1,2,26-Sep-11
2,2 Broke Girls,1,3,3-Oct-11
3,2 Broke Girls,1,4,10-Oct-11
4,2 Broke Girls,1,5,17-Oct-11


In [55]:
len(airdate_data_split.Show.unique())
airdate_data_split.dtypes

Show        object
Season       int32
Episode      int32
Air Date    object
dtype: object

In [51]:
# Merge merged_data and data3_split based on 'Show', 'Season', and 'Episode'
merged_df = pd.merge(nlpdata_imdb, airdate_data_split, on=['Show', 'Season', 'Episode'], how='outer')
merged_df.head()

,Show,Season,Episode,sd_div_mean_1,sd_div_mean_2,sd_div_mean_3,sd_sum_1,sd_sum_2,sd_sum_3,sd_scaled_1,...,Network_HBO (US),Network_IFC,Network_NBC,Network_Paramount Network,Network_Showtime,Network_Starz,Network_The CW,Network_The WB,Network_USA Network,Air Date
0,2 Broke Girls,1,1,5658.359420,9159.676336,4626.990287,18.665142,20.545673,21.697182,598.633647,...,False,False,False,False,False,False,False,False,False,19-Sep-11
1,2 Broke Girls,1,2,6280.502095,3451.804293,7127.226694,19.733889,20.345823,17.440190,629.842904,...,False,False,False,False,False,False,False,False,False,26-Sep-11
2,2 Broke Girls,1,3,3931.604095,3977.461758,5506.511934,21.559420,20.759948,19.837830,694.243568,...,False,False,False,False,False,False,False,False,False,3-Oct-11
3,2 Broke Girls,1,4,5899.921656,7097.733100,3386.857917,21.267363,20.248501,19.434308,684.387650,...,False,False,False,False,False,False,False,False,False,10-Oct-11
4,2 Broke Girls,1,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17-Oct-11


In [111]:
# Function to check for invalid dates
def check_invalid_dates(df_date_column, date_format):
    invalid_indices = []
    for index, date_str in enumerate(df_date_column):
        try:
            pd.to_datetime(date_str, format=date_format)
        except ValueError:
            invalid_indices.append(index)
    return invalid_indices

# Check for invalid dates in 'Air Date' column
invalid_indices = check_invalid_dates(merged_df['Air Date'], '%d-%b-%y')
print(len(invalid_indices), 'rows with invalid \'Air Date\' values')
merged_df['Air Date'].iloc[invalid_indices]

29 rows with invalid 'Air Date' values


889                  Aired in syndication
890                  Aired in syndication
891                  Aired in syndication
1015                 "Career Day, Part 2"
4458                              Unaired
10103                             Unaired
10104                             Unaired
10105                             Unaired
10106                             Unaired
10107                             Unaired
11702                             Unaired
12759                             Unaired
16548                             Unaired
16549                             Unaired
16550                             Unaired
16551                             Unaired
16552                             Unaired
17318                             Unaired
21749                             Unaired
21750                             Unaired
24299                             Unaired
25489    Holly Hester & Apryl Huntzinger 
26703                             Unaired
26704                             

In [112]:
# Drop rows with invalid dates
merged_df.drop(index=invalid_indices, inplace=True)

# Another way to check ivalid and drop
#merged_df['Air Date'] = pd.to_datetime(merged_df['Air Date'], format='%d-%b-%y', errors='coerce')
#merged_df = merged_df.dropna(subset=['Air Date'])

In [113]:
# Convert 'Air Date' column to datetime with correct format
merged_df['Air Date'] = pd.to_datetime(merged_df['Air Date'], format='%d-%b-%y')
merged_df.head()

,Show,Season,Episode,sd_div_mean_1,sd_div_mean_2,sd_div_mean_3,sd_sum_1,sd_sum_2,sd_sum_3,sd_scaled_1,...,Network_HBO (US),Network_IFC,Network_NBC,Network_Paramount Network,Network_Showtime,Network_Starz,Network_The CW,Network_The WB,Network_USA Network,Air Date
0,2 Broke Girls,1,1,5658.359420,9159.676336,4626.990287,18.665142,20.545673,21.697182,598.633647,...,False,False,False,False,False,False,False,False,False,2011-09-19
1,2 Broke Girls,1,2,6280.502095,3451.804293,7127.226694,19.733889,20.345823,17.440190,629.842904,...,False,False,False,False,False,False,False,False,False,2011-09-26
2,2 Broke Girls,1,3,3931.604095,3977.461758,5506.511934,21.559420,20.759948,19.837830,694.243568,...,False,False,False,False,False,False,False,False,False,2011-10-03
3,2 Broke Girls,1,4,5899.921656,7097.733100,3386.857917,21.267363,20.248501,19.434308,684.387650,...,False,False,False,False,False,False,False,False,False,2011-10-10
4,2 Broke Girls,1,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2011-10-17


In [114]:
print(merged_df.shape)
# Drop rows with missing values
merged_df_cleaned = merged_df.dropna()
print(merged_df_cleaned.shape)

(30606, 160)
(705, 160)


In [118]:
# Many missing values in sentiment analysis and Network fields
merged_df.isna().sum().head(20)

Show                 0
Season               0
Episode              0
sd_div_mean_1    29901
sd_div_mean_2    29901
sd_div_mean_3    29901
sd_sum_1         29901
sd_sum_2         29901
sd_sum_3         29901
sd_scaled_1      29901
sd_scaled_2      29901
sd_scaled_3      29901
anger_1          29901
anger_2          29901
anger_3          29901
surprise_1       29901
surprise_2       29901
surprise_3       29901
disgust_1        29901
disgust_2        29901
dtype: int64

In [119]:
merged_df.isna().sum().tail(20)

Network_BBC Two                  29901
Network_CBC Television           29901
Network_CBS                      29901
Network_Cartoon Network          29901
Network_Comedy Central           29901
Network_Disney Channel           29901
Network_FX                       29901
Network_First-run syndication    29901
Network_Fox                      29901
Network_HBO                      29901
Network_HBO (US)                 29901
Network_IFC                      29901
Network_NBC                      29901
Network_Paramount Network        29901
Network_Showtime                 29901
Network_Starz                    29901
Network_The CW                   29901
Network_The WB                   29901
Network_USA Network              29901
Air Date                             0
dtype: int64

In [121]:
# Filter Shows based on minimum number of episodes threshold
min_episodes_threshold = 5  # Adjust as needed

# Calculate number of episodes per show
episode_counts = data.groupby('Show').size()
# Filter out shows with fewer episodes than the threshold
shows_to_keep = episode_counts[episode_counts >= min_episodes_threshold].index

# Filter the original DataFrame based on shows to keep
filtered_data = data[data['Show'].isin(shows_to_keep)]

In [122]:
shows_to_keep

Index(['2 Broke Girls', '24', '30 Rock', '3rd Rock from the Sun',
       '9-1-1: Lone Star', '9/1/2001', 'A Different World',
       'A Million Little Things', 'Abbott Elementary', 'Adventure Time',
       ...
       'Two and a Half Men', 'Ugly Betty', 'Veep', 'Walker, Texas Ranger',
       'Weeds', 'What We Do in the Shadows', 'Without a Trace',
       'Yellowjackets', 'Yellowstone', 'Young Sheldon'],
      dtype='object', name='Show', length=229)

In [129]:
merged_df['Genre'].unique()

array(['Sitcom', nan, 'Serial drama ', 'Sitcom ', 'Procedural drama'],
      dtype=object)

In [58]:
# Step 1: Load the new file
cancelled = pd.read_csv('Cancelled coding - NLP Survival Analysis.csv')
# Step 2: Merge the DataFrames on the 'Show' column
combined_df = pd.merge(merged_df, cancelled, on='Show', how='inner')
# Step 3: Optionally, inspect the merged DataFrame
combined_df.head()

,Show,Season,Episode,sd_div_mean_1,sd_div_mean_2,sd_div_mean_3,sd_sum_1,sd_sum_2,sd_sum_3,sd_scaled_1,...,Network_IFC,Network_NBC,Network_Paramount Network,Network_Showtime,Network_Starz,Network_The CW,Network_The WB,Network_USA Network,Air Date,Cancelled
0,2 Broke Girls,1,1,5658.359420,9159.676336,4626.990287,18.665142,20.545673,21.697182,598.633647,...,False,False,False,False,False,False,False,False,19-Sep-11,1
1,2 Broke Girls,1,2,6280.502095,3451.804293,7127.226694,19.733889,20.345823,17.440190,629.842904,...,False,False,False,False,False,False,False,False,26-Sep-11,1
2,2 Broke Girls,1,3,3931.604095,3977.461758,5506.511934,21.559420,20.759948,19.837830,694.243568,...,False,False,False,False,False,False,False,False,3-Oct-11,1
3,2 Broke Girls,1,4,5899.921656,7097.733100,3386.857917,21.267363,20.248501,19.434308,684.387650,...,False,False,False,False,False,False,False,False,10-Oct-11,1
4,2 Broke Girls,1,6,9570.497331,5883.807213,6103.310464,20.229745,21.176403,21.713179,648.841555,...,False,False,False,False,False,False,False,False,24-Oct-11,1


In [132]:
combined_df['Air Date']

0      2011-09-19
1      2011-09-26
2      2011-10-03
3      2011-10-10
4      2011-10-24
          ...    
700    2023-04-17
701    2023-04-24
702    2023-05-01
703    2023-05-08
704    2023-05-15
Name: Air Date, Length: 705, dtype: object

# TV Series Regression

In [ ]:
#TV Series 1st
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Initialize list to store OLS results
Show_OLS_results_list = []

# TV series regressions
for show in merged_df_cleaned['Show'].unique():
        # Filter data for the current show
    show_data = merged_df_cleaned[merged_df_cleaned['Show'] == show]
    
    # Shift the viewership column to lag it by one episode
    show_data.loc[:, 'Viewership (millions)'] = show_data['Viewership (millions)'].shift(-1)
    
    # Drop the last row since it will have a NaN value in the lagged viewership column
    show_data = show_data.dropna()
    
    # Continue with the rest of your analysis
    if len(show_data) < 5:  # Adjust this number based on your minimum required samples
        print(f"Skipping show '{show}' due to insufficient samples (n={len(show_data)})")
        continue  # Skip to the next show
    
    # Drop unnecessary columns for regression
    X = show_data.drop(["Show", "Season", "Episode", "Viewership (millions)", "Network_USA Network"], axis=1, errors='ignore')
    y = show_data['Viewership (millions)']
    
    # Ensure X contains only numeric columns
    X = X.select_dtypes(include=[np.number])
    
    # Verify if X and y are not empty
    if X.empty or y.empty:
        print(f"Empty data in features or target variable for show '{show}'.")
        continue
   
    # Track feature names before scaling
    original_feature_names = X.columns
    
    # Scale the features
    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X)
    
    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
    
    # Feature selection using RandomForestRegressor
    model_rf = RandomForestRegressor(n_estimators=100, random_state=42)
    
    try:
        model_rf.fit(X_train, y_train)
    except ValueError as e:
        print(f"Error fitting RandomForestRegressor for show '{show}': {e}")
        continue
    
    # Get feature importances and select top 10 features
    feature_importances = model_rf.feature_importances_
    top_indices = np.argsort(feature_importances)[-10:]  # Indices of top 10 features
    selected_features = X.columns[top_indices]
    
    # Select only the important features
    X_selected = X_scaled[:, top_indices]
    
    # Now add the VIF calculation and removal step here
    # Create a DataFrame for the selected features
    X_selected_df = pd.DataFrame(X_selected, columns=selected_features)
    
    # Function to calculate VIF and remove features with VIF > 10
    def calculate_vif(X, threshold=10):
        while True:
            vif_data = pd.DataFrame()
            vif_data["Feature"] = X.columns
            vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
            
            # Check if all VIF values are below the threshold
            max_vif = vif_data["VIF"].max()
            if max_vif < threshold:
                break
            
            # Find the feature with the highest VIF and drop it
            feature_to_drop = vif_data.loc[vif_data["VIF"].idxmax(), "Feature"]
            print(f"Dropping '{feature_to_drop}' with VIF: {max_vif}")
            X = X.drop(columns=[feature_to_drop])
        
        return X, vif_data

    # Apply the VIF function to filter out high VIF features
    X_selected_vif_filtered, vif_data = calculate_vif(X_selected_df)
    
    # If fewer than 10 features remain, re-add features with the lowest VIF or highest importance
    if X_selected_vif_filtered.shape[1] < 10:
        remaining_features_count = X_selected_vif_filtered.shape[1]
        missing_count = 10 - remaining_features_count

        # Find the remaining features from the original top features that were removed
        removed_features = [feature for feature in selected_features if feature not in X_selected_vif_filtered.columns]

        # Sort removed features based on their importance
        sorted_removed_features = sorted(removed_features, key=lambda x: feature_importances[X.columns.get_loc(x)], reverse=True)

        # Re-add features to reach 10
        features_to_add = sorted_removed_features[:missing_count]
        X_selected_vif_filtered = pd.concat([X_selected_vif_filtered, X_selected_df[features_to_add]], axis=1)

    # Update X_selected with filtered features
    X_selected = X_selected_vif_filtered.values
    selected_features = X_selected_vif_filtered.columns
    
    # Calculate correlation matrix with selected features
    correlation_matrix = pd.DataFrame(X_selected, columns=selected_features).corr()
  
    # Perform linear regression with selected features
    X_selected_train, X_selected_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.2, random_state=42)
    X_selected_train = sm.add_constant(X_selected_train)  # Add a constant for the intercept

    model = sm.OLS(y_train, X_selected_train).fit()

    # Extract coefficients, p-values, and other metrics (this section remains the same)
    coefficients = model.params
    p_values = model.pvalues
    
    # Get confidence intervals
    conf = model.conf_int()  # This returns the confidence intervals for all coefficients
    
    # Compute F-statistic manually
    dof_model = X_selected_train.shape[1] - 1  # degrees of freedom of the model
    dof_resid = len(y_train) - X_selected_train.shape[1]  # degrees of freedom of residuals
    ssr = np.sum(model.resid ** 2)  # sum of squared residuals
    mse_resid = ssr / dof_resid  # mean squared error of residuals
    msr = np.sum((model.fittedvalues - np.mean(y_train)) ** 2) / dof_model  # mean squared error of regression
    f_statistic = msr / mse_resid  # F-statistic
    
    # Compute robust standard errors - heteroskedasticity
    robust_results = model.get_robustcov_results(cov_type='HC3')  # HC3 is one of the robust options
    
    # Predictions
    y_pred = model.predict(sm.add_constant(X_selected_test))
    
    # Calculate regression metrics (same as before)
    intercept = model.params[0]
    r_squared = model.rsquared
    adj_r_squared = model.rsquared_adj
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    
    # Cross-Validation
    ols_model = LinearRegression()
    cv_scores = cross_val_score(ols_model, X_selected_vif_filtered, y, cv=5, scoring='r2')
    cv_mean_score = np.mean(cv_scores)
    
    # Create a heatmap for the correlation matrix
    plt.figure(figsize=(10, 8))
    sns.heatmap(correlation_matrix, annot=True, fmt=".2f", cmap="coolwarm",
                xticklabels=correlation_matrix.columns,
                yticklabels=correlation_matrix.columns)
    plt.title(f"Correlation Matrix Heatmap for Show {show}")
    plt.show()
    
    # Display the correlation matrix
    print(f"Correlation matrix for show {show} with selected features:\n")
    print(correlation_matrix)
    print('\n' + '='*80 + '\n')
    
    # Print OLS regression summary
    print(f"OLS Regression results for show '{show}':\n")
    print(model.summary())
    
    # Create a DataFrame for regression results
    results_df = pd.DataFrame({
        'Feature': ['Intercept'] + list(selected_features),  # Include 'Intercept' and selected features
        'Coefficient': coefficients,
        'P-value': p_values,
        'CI Lower Bound': conf[0],
        'CI Upper Bound': conf[1]
    })
    # Append additional regression metrics
    metrics_df = pd.DataFrame({
        'Metric': ['Intercept', 'R-squared', 'Adjusted R-squared', 'Mean Absolute Error', 'Root Mean Square Error', 'F-statistic', 'F-statistic P-value'],
        'Value': [coefficients[0], model.rsquared, model.rsquared_adj, mae, rmse, f_statistic, model.f_pvalue],
})
       
    # Perform cross-validation for regression
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    cv_scores = cross_val_score(LinearRegression(), X_selected, y, cv=kf, scoring='r2')
    
    # Print cross-validation scores
    print(f"Cross-validation scores for show {show}:\n")
    print(cv_scores)
    print(f"Mean CV score: {cv_scores.mean()}")
    print('\n' + '='*80 + '\n')
    from statsmodels.stats.outliers_influence import variance_inflation_factor

    # Calculate VIF for the selected features
    vif_data = pd.DataFrame()
    vif_data["Feature"] = selected_features
    vif_data["VIF"] = [variance_inflation_factor(X_selected, i) for i in range(X_selected.shape[1])]

    # Print regression results
    print(f"Regression results for show {show} with selected features:\n")
    print(results_df.to_string(index=False))
    print('\n' + '='*80 + '\n')

    # Print metrics results
    print(f"Metrics results for show {show} with selected features:\n")
    print(metrics_df.to_string(index=False))
    print('\n' + '='*80 + '\n')
    
    # Store OLS results
    Show_OLS_results_list.append({
        'Show': show,
        'OLS_R_squared': r_squared,
        'OLS_Adj_R_squared': adj_r_squared,
        'OLS_MAE': mae,
        'OLS_RMSE': rmse,
        'CV_Mean_Score': cv_scores.mean()
    })

# Convert results to DataFrame and save to Excel
results_df = pd.DataFrame(Show_OLS_results_list)
results_df.to_excel('Show_OLS_results_list.xlsx', index=False)